## Imports

In [1]:
import pandas as pd
import numpy as np

In [2]:
# Load the provided CSV file
file_path = '../Dataset/Physiology.csv'
data = pd.read_csv(file_path)
data

,patient_id,date,device_type,value,unit
0,0697d,2019-06-28 13:42:09,Body Temperature,36.072,Cel
1,0697d,2019-06-28 13:42:44,Body Temperature,35.831,Cel
2,0697d,2019-06-28 13:43:40,Body Temperature,35.831,Cel
3,0697d,2019-06-28 13:45:15,Systolic blood pressure,165.000,mm[Hg]
4,0697d,2019-06-28 13:45:15,Diastolic blood pressure,82.000,mm[Hg]
...,...,...,...,...,...
17674,fd100,2019-06-28 21:09:27,Heart rate,61.000,beats/min
17675,fd100,2019-06-28 21:32:19,Body Temperature,36.879,Cel
17676,fd100,2019-06-28 21:32:46,Body Temperature,36.879,Cel
17677,fd100,2019-06-29 21:05:11,Body Temperature,36.240,Cel


In [3]:
data['date'] = pd.to_datetime(data['date'])
data.dtypes

patient_id             object
date           datetime64[ns]
device_type            object
value                 float64
unit                   object
dtype: object

In [8]:
"""
IGNORE FOR NOW

# Separate the date and time into two columns
data[['Date', 'Time']] = data['date'].str.split(' ', expand=True)

# Exclude rows where device_type is 'Body Weight', 'Skin Temperature', or 'Total Body Water'
filtered_data = data[~data['device_type'].isin(['Body Weight', 'Skin Temperature', 'Total Body Water'])]
filtered_data = filtered_data.drop(['date'], axis=1)
filtered_data['DateTime'] = pd.to_datetime(filtered_data_updated['Date'] + ' ' + filtered_data_updated['Time'])

filtered_data.head()

"""

,patient_id,device_type,value,unit,Date,Time
0,0697d,Body Temperature,36.072,Cel,2019-06-28,13:42:09
1,0697d,Body Temperature,35.831,Cel,2019-06-28,13:42:44
2,0697d,Body Temperature,35.831,Cel,2019-06-28,13:43:40
3,0697d,Systolic blood pressure,165.000,mm[Hg],2019-06-28,13:45:15
4,0697d,Diastolic blood pressure,82.000,mm[Hg],2019-06-28,13:45:15


In [4]:
# Exclude rows where device_type is 'Body Weight', 'Skin Temperature', or 'Total Body Water'
data = data[~data['device_type'].isin(['O/E - muscle mass', 'Skin Temperature', 'Total body water'])]

In [5]:
def assign_time_window(row):
    # Extracting hour and minute from the datetime
    hour = row.hour
    minute = row.minute

    # The first window is based on the hour itself
    start_hour_1 = hour
    end_hour_1 = (start_hour_1 + 1) % 24  # Wrapping around midnight

    # The second window depends on whether the minute is >= 30
    if minute >= 30:
        start_hour_2 = hour
    else:
        start_hour_2 = hour - 1 if hour > 0 else 23  # Wrapping around midnight

    end_hour_2 = (start_hour_2 + 1) % 24

    # Formatting the time windows
    window1 = f"{start_hour_1:02d}:00-{end_hour_1:02d}:00"
    window2 = f"{start_hour_2:02d}:30-{end_hour_2:02d}:30"

    return [window1, window2]

# Applying the function to each row and expanding the results into two columns
data[['window_1', 'window_2']] = pd.DataFrame(data['date'].apply(assign_time_window).tolist(), index=data.index)

data.head()

C:\Users\youss\AppData\Local\Temp\ipykernel_4392\2215228946.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[['window_1', 'window_2']] = pd.DataFrame(data['date'].apply(assign_time_window).tolist(), index=data.index)
C:\Users\youss\AppData\Local\Temp\ipykernel_4392\2215228946.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[['window_1', 'window_2']] = pd.DataFrame(data['date'].apply(assign_time_window).tolist(), index=data.index)


,patient_id,date,device_type,value,unit,window_1,window_2
0,0697d,2019-06-28 13:42:09,Body Temperature,36.072,Cel,13:00-14:00,13:30-14:30
1,0697d,2019-06-28 13:42:44,Body Temperature,35.831,Cel,13:00-14:00,13:30-14:30
2,0697d,2019-06-28 13:43:40,Body Temperature,35.831,Cel,13:00-14:00,13:30-14:30
3,0697d,2019-06-28 13:45:15,Systolic blood pressure,165.000,mm[Hg],13:00-14:00,13:30-14:30
4,0697d,2019-06-28 13:45:15,Diastolic blood pressure,82.000,mm[Hg],13:00-14:00,13:30-14:30


In [6]:
# Function to convert window and date into a datetime object with the start time of the window
def window_to_datetime(row, window_col):
    # Extracting date and start time from the row
    date = row['date'].date()
    start_time = row[window_col].split('-')[0]

    # Combining date and start time into a datetime object
    return pd.to_datetime(f"{date} {start_time}")

# Applying the function to convert window_1 and window_2
data['window_1'] = data.apply(window_to_datetime, window_col='window_1', axis=1)
data['window_2'] = data.apply(window_to_datetime, window_col='window_2', axis=1)

# Display the first 20 rows with the updated columns
data.head(20)

C:\Users\youss\AppData\Local\Temp\ipykernel_4392\2918942288.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['window_1'] = data.apply(window_to_datetime, window_col='window_1', axis=1)
C:\Users\youss\AppData\Local\Temp\ipykernel_4392\2918942288.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['window_2'] = data.apply(window_to_datetime, window_col='window_2', axis=1)


,patient_id,date,device_type,value,unit,window_1,window_2
0,0697d,2019-06-28 13:42:09,Body Temperature,36.072,Cel,2019-06-28 13:00:00,2019-06-28 13:30:00
1,0697d,2019-06-28 13:42:44,Body Temperature,35.831,Cel,2019-06-28 13:00:00,2019-06-28 13:30:00
2,0697d,2019-06-28 13:43:40,Body Temperature,35.831,Cel,2019-06-28 13:00:00,2019-06-28 13:30:00
3,0697d,2019-06-28 13:45:15,Systolic blood pressure,165.000,mm[Hg],2019-06-28 13:00:00,2019-06-28 13:30:00
4,0697d,2019-06-28 13:45:15,Diastolic blood pressure,82.000,mm[Hg],2019-06-28 13:00:00,2019-06-28 13:30:00
5,0697d,2019-06-28 13:45:15,Heart rate,42.000,beats/min,2019-06-28 13:00:00,2019-06-28 13:30:00
6,0697d,2019-06-28 14:25:07,Body Temperature,36.599,Cel,2019-06-28 14:00:00,2019-06-28 13:30:00
7,0697d,2019-06-28 14:25:37,Body Temperature,36.599,Cel,2019-06-28 14:00:00,2019-06-28 13:30:00
8,0697d,2019-06-28 14:40:13,Body weight,86.300,kg,2019-06-28 14:00:00,2019-06-28 14:30:00
11,0697d,2019-06-29 17:28:50,Body Temperature,37.257,Cel,2019-06-29 17:00:00,2019-06-29 16:30:00


In [8]:
data.to_csv('../Dataset/Physiology_cleaned_V2.csv', index=False)

In [9]:
# Ignore for now

# # Convert 'date' to datetime format
# data['date'] = pd.to_datetime(data['date'])

# data_2 = data.copy()

# # Create a function to map device_type to the corresponding column name in the final dataframe
# def map_device_type_to_column(device_type):
#     mapping = {
#         'Body Temperature': 'Body Temperature',
#         'Systolic blood pressure': 'Systolic Blood Pressure',
#         'Diastolic blood pressure': 'Diastolic Blood Pressure',
#         'Heart Rate': 'Heart Rate',
#         'Body weight': 'Body weight'
#     }
#     return mapping.get(device_type, None)

# # Apply the mapping
# data_2['Measurement'] = data_2['device_type'].apply(map_device_type_to_column)

# # Remove rows with measurements that don't match the desired columns (if any)
# data_2 = data_2.dropna(subset=['Measurement'])

# # For each patient, device_type, and window, calculate the average value
# averages_window_1 = data_2.groupby(['patient_id', 'Measurement', 'window_1'])['value'].mean().reset_index()
# averages_window_2 = data_2.groupby(['patient_id', 'Measurement', 'window_2'])['value'].mean().reset_index()

# # Rename columns for merging
# averages_window_1.rename(columns={'window_1': 'Window', 'value': 'Average'}, inplace=True)
# averages_window_2.rename(columns={'window_2': 'Window', 'value': 'Average'}, inplace=True)

# # Combine the two sets of averages
# averages_combined = pd.concat([averages_window_1, averages_window_2])

# # Pivot the table to have one row per patient_id and window, and columns for each measurement
# final_data = averages_combined.pivot_table(index=['patient_id', 'Window'], columns='Measurement', values='Average').reset_index()

# # Rename the columns as specified
# # final_data.columns = ['Patient ID', 'Window', 'Body Temperature', 'Diastolic Blood Pressure', 'Heart Rate', 'O/E - Muscle Mass', 'Systolic Blood Pressure']

# # Display the first few rows of the final dataframe
# final_data.head()


C:\Users\youss\AppData\Local\Temp\ipykernel_4392\2743704572.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['date'] = pd.to_datetime(data['date'])


Measurement,patient_id,Window,Body Temperature,Body weight,Diastolic Blood Pressure,Systolic Blood Pressure
0,0697d,2019-06-28 13:00:00,35.911333,NaN,82.0,165.0
1,0697d,2019-06-28 13:30:00,36.186400,NaN,82.0,165.0
2,0697d,2019-06-28 14:00:00,36.599000,86.3,NaN,NaN
3,0697d,2019-06-28 14:30:00,NaN,86.3,NaN,NaN
4,0697d,2019-06-29 16:30:00,37.257000,NaN,NaN,NaN


In [14]:
# Convert 'date' to datetime format
data = pd.read_csv('../Dataset/Physiology_cleaned_V2.csv')
data['date'] = pd.to_datetime(data['date'])

data_2 = data.copy()

# Adjust the mapping function again to account for the correct labeling of "O/E - muscle mass"
def final_map_device_type_to_column(device_type):
    mapping = {
        'Body Temperature': 'Body Temperature',
        'Systolic blood pressure': 'Systolic Blood Pressure',
        'Diastolic blood pressure': 'Diastolic Blood Pressure',
        'Heart rate': 'Heart Rate',
        'Body weight': 'Body weight'
    }
    return mapping.get(device_type, None)

# Apply the final corrected mapping to the dataset
data_2['Final Measurement'] = data_2['device_type'].apply(final_map_device_type_to_column)

# Recalculate averages with the final measurement mapping, correctly identifying all measurements
final_averages_window_1 = data_2.groupby(['patient_id', 'Final Measurement', 'window_1'])['value'].mean().reset_index()
final_averages_window_2 = data_2.groupby(['patient_id', 'Final Measurement', 'window_2'])['value'].mean().reset_index()

# Rename columns for merging
final_averages_window_1.rename(columns={'window_1': 'Window', 'value': 'Average'}, inplace=True)
final_averages_window_2.rename(columns={'window_2': 'Window', 'value': 'Average'}, inplace=True)

# Combine the two sets of averages
final_averages_combined = pd.concat([final_averages_window_1, final_averages_window_2])

# Pivot the table to have one row per patient_id and window, and columns for each measurement
final_corrected_data = final_averages_combined.pivot_table(index=['patient_id', 'Window'], columns='Final Measurement', values='Average', aggfunc='first').reset_index()

# Ensure all expected columns are present
final_expected_columns = ['patient_id', 'Window', 'Body Temperature', 'Diastolic Blood Pressure', 'Heart Rate', 'Body weight', 'Systolic Blood Pressure']
final_corrected_data = final_corrected_data.reindex(columns=final_expected_columns)

In [15]:
final_corrected_data.head()
final_corrected_data.tail()

Final Measurement,patient_id,Window,Body Temperature,Diastolic Blood Pressure,Heart Rate,Body weight,Systolic Blood Pressure
7626,fd100,2019-06-28 20:30:00,36.6115,74.0,61.0,NaN,138.0
7627,fd100,2019-06-28 21:00:00,36.5380,74.0,61.0,NaN,138.0
7628,fd100,2019-06-28 21:30:00,36.8790,NaN,NaN,NaN,NaN
7629,fd100,2019-06-29 20:30:00,36.2400,NaN,NaN,NaN,NaN
7630,fd100,2019-06-29 21:00:00,36.2400,NaN,NaN,NaN,NaN


In [16]:
final_corrected_data.shape

(7631, 7)

In [18]:
# First, correct the attempt to view the agitation_labels data to understand its structure for further processing.
# Given the previous issues, we'll proceed with the described logic to add an "Agitation" column to the measurements dataset.

# Assuming agitation_labels has been correctly filtered for "Agitation" types:
# - Convert dates in agitation_labels to ensure datetime format
# - Apply the logic for determining which time windows to flag for agitation

# Reattempt to load and filter the labels dataset for "Agitation" directly here for completeness
labels_path = '..\Dataset\Labels.csv'
labels_data = pd.read_csv(labels_path)
labels_data['date'] = pd.to_datetime(labels_data['date'])
agitation_labels = labels_data[labels_data['type'] == 'Agitation']

# Define a function to apply the described logic for setting the "Agitation" column based on the rules provided
def apply_agitation_flag(row, agitation_labels):
    # Filter agitation_labels for the patient in the row
    patient_agitations = agitation_labels[agitation_labels['patient_id'] == row['patient_id']]
    
    # Check for any agitation events that affect the current row's window
    for _, agitation in patient_agitations.iterrows():
        window_start = pd.to_datetime(row['Window'])
        window_end = window_start + pd.Timedelta(hours=1)  # Assuming each window is 1 hour
        
        # Check if agitation is logged at 12pm or 6pm
        if agitation['date'].hour == 18:  # 6pm
            if 12 <= window_start.hour < 18:
                return 1
        elif agitation['date'].hour == 12:  # 12pm
            if window_start.hour < 12 or window_start.hour >= 18:
                return 1
                
    # If no conditions met, return 0
    return 0

# Apply the function to the final_corrected_data dataframe to create the "Agitation" column
final_corrected_data['Agitation'] = final_corrected_data.apply(lambda row: apply_agitation_flag(row, agitation_labels), axis=1)

# Display the first few rows of the dataframe with the new "Agitation" column
final_corrected_data.head()


Final Measurement,patient_id,Window,Body Temperature,Diastolic Blood Pressure,Heart Rate,Body weight,Systolic Blood Pressure,Agitation
0,0697d,2019-06-28 13:00:00,35.911333,82.0,42.0,NaN,165.0,0
1,0697d,2019-06-28 13:30:00,36.186400,82.0,42.0,NaN,165.0,0
2,0697d,2019-06-28 14:00:00,36.599000,NaN,NaN,86.3,NaN,0
3,0697d,2019-06-28 14:30:00,NaN,NaN,NaN,86.3,NaN,0
4,0697d,2019-06-29 16:30:00,37.257000,NaN,NaN,NaN,NaN,0


In [19]:
final_corrected_data[final_corrected_data["Agitation"] == 1]

Final Measurement,patient_id,Window,Body Temperature,Diastolic Blood Pressure,Heart Rate,Body weight,Systolic Blood Pressure,Agitation
9,099bc,2019-05-15 11:00:00,NaN,NaN,NaN,53.3,NaN,1
10,099bc,2019-05-15 11:30:00,37.229286,88.0,66.0,53.3,141.0,1
13,099bc,2019-05-16 08:30:00,37.045000,78.0,71.0,52.7,152.0,1
14,099bc,2019-05-16 09:00:00,36.794857,78.0,71.0,52.7,152.0,1
15,099bc,2019-05-16 09:30:00,36.753167,NaN,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...
7505,eca1f,2019-06-26 09:30:00,36.766000,59.0,87.0,66.0,103.0,1
7506,eca1f,2019-06-27 09:00:00,36.502000,63.0,89.0,65.3,111.0,1
7507,eca1f,2019-06-27 09:30:00,36.502000,63.0,89.0,65.3,111.0,1
7508,eca1f,2019-06-28 09:30:00,36.322000,68.0,91.0,65.8,107.0,1


In [20]:
final_corrected_data.to_csv('../Dataset/Physiology_Formatted_Dataset_V22.csv', index=False)

In [23]:
df = pd.read_csv('../Dataset/Physiology_Formatted_Dataset_V22.csv')
df.head()

df['Window'] = pd.to_datetime(df['Window'])
df['day'] = df['Window'].dt.date

In [24]:
# Sort your data
df.sort_values(by=['patient_id', 'Window'], inplace=True)

def locf_with_agitation(df):
    # Identify the columns to fill
    columns_to_fill = [col for col in df.columns if col not in ['patient_id', 'Window', 'Agitation','day']]
    
    for col in columns_to_fill:
        # Apply forward fill where 'Agitation' value is the same as in the next row
        df[col] = df.groupby(['patient_id', 'day', 'Agitation'])[col].ffill()

    return df

data_filled = df.groupby('patient_id').apply(locf_with_agitation)
data_filled.reset_index(drop=True, inplace=True)
data_filled

,patient_id,Window,Body Temperature,Diastolic Blood Pressure,Heart Rate,Body weight,Systolic Blood Pressure,Agitation,day
0,0697d,2019-06-28 13:00:00,35.911333,82.0,42.0,NaN,165.0,0,2019-06-28
1,0697d,2019-06-28 13:30:00,36.186400,82.0,42.0,NaN,165.0,0,2019-06-28
2,0697d,2019-06-28 14:00:00,36.599000,82.0,42.0,86.3,165.0,0,2019-06-28
3,0697d,2019-06-28 14:30:00,36.599000,82.0,42.0,86.3,165.0,0,2019-06-28
4,0697d,2019-06-29 16:30:00,37.257000,NaN,NaN,NaN,NaN,0,2019-06-29
...,...,...,...,...,...,...,...,...,...
7626,fd100,2019-06-28 20:30:00,36.611500,74.0,61.0,NaN,138.0,0,2019-06-28
7627,fd100,2019-06-28 21:00:00,36.538000,74.0,61.0,NaN,138.0,0,2019-06-28
7628,fd100,2019-06-28 21:30:00,36.879000,74.0,61.0,NaN,138.0,0,2019-06-28
7629,fd100,2019-06-29 20:30:00,36.240000,NaN,NaN,NaN,NaN,0,2019-06-29


In [25]:
data_filled.isna().sum()

patient_id                     0
Window                         0
Body Temperature             917
Diastolic Blood Pressure    3256
Heart Rate                  3256
Body weight                 3129
Systolic Blood Pressure     3256
Agitation                      0
day                            0
dtype: int64

In [28]:
data_filled = data_filled.dropna()
data_filled[data_filled['Agitation'] == 1]

,patient_id,Window,Body Temperature,Diastolic Blood Pressure,Heart Rate,Body weight,Systolic Blood Pressure,Agitation,day
10,099bc,2019-05-15 11:30:00,37.229286,88.0,66.0,53.3,141.0,1,2019-05-15
13,099bc,2019-05-16 08:30:00,37.045000,78.0,71.0,52.7,152.0,1,2019-05-16
14,099bc,2019-05-16 09:00:00,36.794857,78.0,71.0,52.7,152.0,1,2019-05-16
15,099bc,2019-05-16 09:30:00,36.753167,78.0,71.0,52.7,152.0,1,2019-05-16
75,099bc,2019-06-05 10:30:00,35.987900,78.0,55.0,52.1,159.0,1,2019-06-05
...,...,...,...,...,...,...,...,...,...
7505,eca1f,2019-06-26 09:30:00,36.766000,59.0,87.0,66.0,103.0,1,2019-06-26
7506,eca1f,2019-06-27 09:00:00,36.502000,63.0,89.0,65.3,111.0,1,2019-06-27
7507,eca1f,2019-06-27 09:30:00,36.502000,63.0,89.0,65.3,111.0,1,2019-06-27
7508,eca1f,2019-06-28 09:30:00,36.322000,68.0,91.0,65.8,107.0,1,2019-06-28


In [29]:
data_filled.to_csv('../Dataset/Physiology_Formatted_Dataset_v222.csv', index=False)

In [34]:
data_filled

,patient_id,Window,Body Temperature,Diastolic Blood Pressure,Heart Rate,Body weight,Systolic Blood Pressure,Agitation,day
2,0697d,2019-06-28 14:00:00,36.599000,82.0,42.0,86.3,165.0,0,2019-06-28
3,0697d,2019-06-28 14:30:00,36.599000,82.0,42.0,86.3,165.0,0,2019-06-28
5,0697d,2019-06-29 17:00:00,36.914833,78.5,60.0,86.1,157.0,0,2019-06-29
6,0697d,2019-06-29 17:30:00,36.743750,78.5,60.0,86.1,157.0,0,2019-06-29
10,099bc,2019-05-15 11:30:00,37.229286,88.0,66.0,53.3,141.0,1,2019-05-15
...,...,...,...,...,...,...,...,...,...
7509,eca1f,2019-06-28 10:00:00,36.322000,68.0,91.0,65.8,107.0,1,2019-06-28
7510,eca1f,2019-06-30 14:00:00,36.934000,71.0,87.0,65.7,129.0,0,2019-06-30
7511,eca1f,2019-06-30 14:30:00,36.934000,71.0,87.0,65.7,129.0,0,2019-06-30
7623,fd100,2019-06-27 11:00:00,36.699600,76.0,54.0,55.8,153.0,0,2019-06-27


In [35]:
data_filled.drop(['patient_id','Window','day'],axis=1).corr()

,Body Temperature,Diastolic Blood Pressure,Heart Rate,Body weight,Systolic Blood Pressure,Agitation
Body Temperature,1.000000,-0.132654,-0.009144,-0.088304,-0.133055,-0.043875
Diastolic Blood Pressure,-0.132654,1.000000,0.078680,-0.052179,0.726111,0.071063
Heart Rate,-0.009144,0.078680,1.000000,-0.101776,-0.057234,0.055022
Body weight,-0.088304,-0.052179,-0.101776,1.000000,-0.043054,0.235178
Systolic Blood Pressure,-0.133055,0.726111,-0.057234,-0.043054,1.000000,0.072483
Agitation,-0.043875,0.071063,0.055022,0.235178,0.072483,1.000000
